<a href="https://colab.research.google.com/github/anhquan-truong/PM520/blob/main/HW/PM520_HW1_AnhQuanTRUONG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1. Linear regression and normal equations

In [5]:
import jax
import jax.numpy as jnp
import jax.random as rdm
import jax.numpy.linalg as jnpla
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# 1. Linear model simulation
In class we defined a Python function that simulates $N$ $P\times 1$ variables $X$ (i.e. an $N \times P$ matrix $X$) and outcome $y$ as a linear function of $X$. Please include its definition here and use for problem 2.

Given $N \times P$ matrix $X$, a $P \times 1$ vector $\beta$, and $N \times 1$ outcome vector $y$, and a random variabl $\epsilon$, with $\mathbb{E}[\epsilon]=0$ and $Var(\epsilon) = \sigma^2$. Then, we can describe the $y$ as a linear function of $X$ as

$$
y = X\times \beta + \epsilon\\
$$

In [8]:
def sim_linear_reg(key, N, P, r2=0.5):
  key, b_key = rdm.split(key)
  b = rdm.normal(b_key, shape=(P,))

  key, X_key = rdm.split(key)
  X = rdm.normal(X_key, shape = (N, P))

  y_hat = X @ b # this is the predicted y without error eps
  s2pred = jnp.var(y_hat)
  s2tot = s2pred/r2 # s2tot = s2pred + s2e
  s2e =( s2tot - s2pred)
  # or s2e = s2pred * (1/r2-1)

  key, e_key = rdm.split(key)
  eps = rdm.normal(e_key, shape = (N,)) * jnp.sqrt(s2e)
  y = y_hat + eps
  return y, y_hat, b, X, eps

seed = 912
key = rdm.PRNGKey(seed) # creating key from seed

N, P = 1000, 5 # a matrix 1000 x 150

y, y_hat, b, X, eps = sim_linear_reg(key, N, P, r2=0.5)

#Double check eps
print(f"Mean of epsilon: {jnp.mean(eps)}\n Var of epsilon: {jnp.var(eps)}")

Mean of epsilon: 0.9396470785140991
 Var of epsilon: 158.33473205566406


# 2. Just-in time decorator and ordinary least squares
Complete the definition of `ordinary_least_squares` below, that estimates the effect and its standard error. `@jit` wraps a function to perform just-in-time compilation, which boosts computational performance/speed.

Compare the times of with and without JIT
Hint: use [`block_until_ready()`](https://jax.readthedocs.io/en/latest/_autosummary/jax.block_until_ready.html) to get correct timing estimates.

In [7]:
import jax

from jax import jit

def ordinary_least_squares(X, y):
  N, P = X.shape
  XtX = X.T @ X
  inv_XtX = jnpla.inv(XtX) # Calculate inverse once
  b_hat = inv_XtX @ (X.T@y)

  s2_e = jnp.sum(jnp.square(y - X @ b_hat)) / (N - P)
  # Corrected se_b_hat calculation: sqrt of diagonal of (s2_e * (X'X)^-1)
  se_b_hat = jnp.sqrt(jnp.diag(s2_e * inv_XtX))
  return b_hat, se_b_hat

jit_ordinary_least_squares = jit(ordinary_least_squares)

N, P = 1000, 150 # a matrix 1000 x 150
y, y_hat, b, X, eps = sim_linear_reg(key, N, P, r2=0.5)

b_hat_dir, se_b_hat = jit_ordinary_least_squares(X,y)


# Double check
model = LinearRegression(fit_intercept=False).fit(X, y)
b_hat_linreg = model.coef_
print(f'b_hat direct and b_hat linreg the same? {jnp.allclose(b_hat_dir,b_hat_linreg, atol=1e-5)}')

%timeit ordinary_least_squares(X,y)[0].block_until_ready()
%timeit jit_ordinary_least_squares(X,y)[0].block_until_ready()

b_hat direct and b_hat linreg the same? True
4.32 ms ± 371 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.7 ms ± 256 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 3. OLS derivation
Assume that $y = X \beta + \epsilon$ where $y$ is $N \times 1$ vector, $X$ is an $N \times P$ matrix where $P < N$ and $\epsilon$ is a random variable such that $\mathbb{E}[\epsilon_i] = 0$ and $\mathbb{V}[\epsilon_i] = \sigma^2$ for all $i = 1 \dots n$. Derive the OLS "normal equations".

The goal is to find $\beta$ such that the residual sum of square is minimal (RSS), or the sun of square of $\epsilon$ is minimal. The $RSS(\beta)$ is

$$RSS(\beta)=\sum_{i=1}^n (y_i - x_i^T\beta)^2$$

We want to find

$$\beta^*=argmin \ RSS(\beta)$$

**Approach**: We find stationary point, i.e. the point with zero gradients. We take the derivative of $RSS(\beta)$ with respect to $\beta$

$$
\begin{align*}
\frac{\partial RSS(\beta)}{\partial \beta} &= 2\sum_{i=1}^n (x_i^T\beta-y_i)x_i \\
&=2\sum_{i=1}^n (x_i (x_i^T \beta) - x_i y_i) \\
&=2\sum_{i=1}^n (x_i x_i^T) \beta - 2\sum_{i=1}^n (x_i y_i) \\
&=2[(X^T X) \beta - (X^T Y)] \\
\end{align*}
$$

From that we have

$$
\begin{align*}
\nabla RSS(\beta) &= 0 \iff 2[(X^T X) \hat{\beta} - (X^T Y)]  = 0 \iff \hat{\beta} = (X^T X)^{-1} (X^T Y)\\
\end{align*}
$$

assuming $(X^T X)$ is invertible.

Now we calculate the variance of $\hat{\beta}$

From above, we have

$$
\begin{align*}
\hat{\beta} &= (X^T X)^{-1} (X^T Y) \\
&= (X^T X)^{-1} X^T (X \beta + \epsilon)\\
&= (X^T X)^{-1} X^T X \beta + (X^T X)^{-1} X^T \epsilon\\
&= I_n \beta + (X^T X)^{-1} X^T \epsilon\\
\Rightarrow \beta - \hat{\beta} &= (X^T X)^{-1} X^T \epsilon\\
\Rightarrow Var(\hat{\beta}) &= Var[(X^T X)^{-1} X^T \epsilon]\\\\
\end{align*}
$$

Apply variances rule for a matrix-vector mulilplication $Var(Ax) = AVar(X)A^T$, we have

$$
\begin{align*}
Var(\hat{\beta}) &= [(X^T X)^{-1} X^T]\ Var(\epsilon)\ [(X^T X)^{-1} X^T]^T\\
&= [(X^T X)^{-1} X^T]\  Var(\epsilon)\ X [(X^T X)^{-1}]^T\\
&= [(X^T X)^{-1} X^T]\ Var(\epsilon)\ X [(X^T X)^T]^{-1}\\\\
\Rightarrow Var(\hat{\beta}) &= [(X^T X)^{-1} X^T]\ Var(\epsilon)\ X (X^T X)^{-1}\\
 &= [(X^T X)^{-1} X^T]\ σ^2\ X (X^T X)^{-1}\\
 &= σ^2\ [(X^T X)^{-1} X^T X] (X^T X)^{-1}\\
&= σ^2\ (X^T X)^{-1}\\
\end{align*}
$$
Assuming errors are independent and homoskedasticity